In [1]:
import yaml

In [2]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
from pathlib import Path
import requests
from pprint import pprint
from typing import NamedTuple, List
import pickle
import time
from dotenv import load_dotenv
import os
load_dotenv()

False

In [3]:
class Card(NamedTuple):
    src: List[str]
    title: str

## Getting One Page


In [14]:
with open("source.yaml","r") as fp:
    source = yaml.safe_load(fp)


URL = source["source"]
page_number = 1
set_res = requests.get(
    URL.format(page_number=page_number))
soup = BeautifulSoup(set_res.text, 'html.parser')
card_imgs = soup.find_all('img', {'class': 'card-image'})

In [15]:
import re
src_sets = [Card(    [re.sub(r' \d\d\dw',"",j) for j in i['srcset'].split(', ')]     , i.parent['title'])
            for i in card_imgs]

card_set = [Card(card.src[1].replace(" 225w", "").strip(), card.title)
            for card in src_sets]


In [16]:
src_sets[0].src

['https://pkmncards.com/wp-content/uploads/svbsp_en_162_std.jpg',
 'https://pkmncards.com/wp-content/uploads/svbsp_en_162_std-225x314.jpg',
 'https://pkmncards.com/wp-content/uploads/svbsp_en_162_std-312x436.jpg']

In [22]:
data_path = (Path.cwd()/'data')
data_path.mkdir(parents=True, exist_ok=True)

try:
    with open(data_path / 'cards.pickle', 'rb') as fp:
        cards = pickle.load(fp)
except EOFError:
    cards = []
except:
    cards = []

try:
    with open(data_path / 'saved_index.pickle', 'rb') as fp:
         START = int(pickle.load(fp)) + 1

except:
        START = 1

FETCH_NUMBER = 5
for page in range(START, START+FETCH_NUMBER):
    BASE_URL = source["source"]
    try:
        set_res = requests.get(BASE_URL.format(page_number=page))
        soup = BeautifulSoup(set_res.text, 'html.parser')
        card_imgs = soup.find_all('img', {'class': 'card-image'})
        src_sets = [Card(i['srcset'].split(', '), i.parent['title'])
                    for i in card_imgs]
        card_set = [Card(card.src[1].replace(" 225w", "").strip(),
                         card.title) for card in src_sets]

        cards.extend(card_set)

        if (page % 5 == 0):
            with open(data_path / 'cards.pickle', 'wb') as fp:
                pickle.dump(cards, fp)
            with open(data_path / 'saved_index.pickle', 'wb') as fp:
                pickle.dump(page, fp)
        print(page)
    except:
        pass

    time.sleep(0.1)

print(len(cards))
# pprint(cards)

26
27
28
29
30
1740
